Monte Carlo (M=30) for Unsupervised K-Means

Everything same as before, only thing here
Dataset has been once WHOLE Normalized

So this is for the Normalized Dataset

All coding portion same, only data has been NORMALIZED once, whole, in an 'as is given' fashion, as before

In [12]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import math
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import normalize




df = pd.read_csv('wdbc.csv', header = None)


output = df.iloc[:,0]


for i in np.arange(0,df.shape[0]):
    if output.iloc[i] == 'B':
        output.iloc[i] = 0
    else:
        output.iloc[i] = 1



features = df.iloc[:,1:]




features_normal = normalize(features)


features_normal_df = pd.DataFrame(features_normal)


norm_whole = pd.concat([features_normal_df,output],axis=1)


col_head = norm_whole.columns


norm_whole.columns= ['X1','X2','X3','X4','X5','X6','X7','X8','X9','X10','X11','X12','X13','X14','X15','X16','X17','X18','X19','X20','X21','X22','X23','X24','X25','X26','X27','X28','X29','X30','y']


norm_whole.sort_values(by=['y'],inplace=True)


norm_whole.reset_index(drop=True,inplace=True)


X_y_0 = norm_whole.iloc[0:357,:]
X_y_1 = norm_whole.iloc[357:569,:]
X_y_1.reset_index(drop=True,inplace=True)



Precision_Train = pd.DataFrame(index=np.arange(1,31),columns=['Precision Train'])

Recall_Train = pd.DataFrame(index=np.arange(1,31),columns=['Recall Train'])

F1_Train = pd.DataFrame(index=np.arange(1,31),columns=['F1 Train'])

Accuracy_Train = pd.DataFrame(index=np.arange(1,31),columns=['Accuracy Train'])

AUC_Train = pd.DataFrame(index=np.arange(1,31),columns=['AUC Train'])

Train_Error = pd.DataFrame(index=np.arange(1,31),columns=['Train Error'])





Precision_Test = pd.DataFrame(index=np.arange(1,31),columns=['Precision Test'])

Recall_Test = pd.DataFrame(index=np.arange(1,31),columns=['Recall Test'])

F1_Test = pd.DataFrame(index=np.arange(1,31),columns=['F1 Test'])

Accuracy_Test = pd.DataFrame(index=np.arange(1,31),columns=['Accuracy Test'])

AUC_Test = pd.DataFrame(index=np.arange(1,31),columns=['AUC Test'])

Test_Error = pd.DataFrame(index=np.arange(1,31),columns=['Test Error'])


# NOTE : No C here. Also CV to estimate test errors hasn't been done as it isn't needed here. Could be easily performed
# but increases computational time, as such. 





for mc in np.arange(0,30):
    
    print(mc)         # for status check


    X_y_train_0,X_y_test_0 = train_test_split(X_y_0,test_size = 0.2,shuffle=True)
    X_y_train_1,X_y_test_1 = train_test_split(X_y_1,test_size = 0.2,shuffle=True)



    X_y_train = pd.concat([X_y_train_0,X_y_train_1],axis=0)
    X_y_test = pd.concat([X_y_test_0,X_y_test_1],axis=0)



    X_y_train = shuffle(X_y_train)
    X_y_test = shuffle(X_y_test)



    X_y_train.reset_index(drop=True,inplace=True)
    X_y_test.reset_index(drop=True,inplace=True)


    X_train = X_y_train.drop(columns=['y'])
    X_test = X_y_test.drop(columns=['y'])
    y_train = X_y_train['y'].astype(int)
    y_test =  X_y_test['y'].astype(int)
 






    for i in np.arange(0,10):        
        kmeans = KMeans(n_clusters=2,init='random',n_init=10).fit(X_train)   


    y_train_pred_cluster = kmeans.labels_
    cluster_centers = kmeans.cluster_centers_

  








    X_train_dist_0 = kmeans.transform(X_train)[:,0]
    X_train_dist_1 = kmeans.transform(X_train)[:,1]



    all_indx = np.arange(0,X_train.shape[0])          
    list_cluster_0 = []        
    list_cluster_1 = []         

    for i in all_indx:
        if y_train_pred_cluster[i] == 0:
            list_cluster_0.append(i)
        else:
            list_cluster_1.append(i)



    dist_0_0 = X_train_dist_0[list_cluster_0]  
    dist_1_1 = X_train_dist_1[list_cluster_1]   



    df_0 = pd.DataFrame(np.transpose(np.array([np.array(list_cluster_0),dist_0_0])),columns=['index','distance'])
    df_1 = pd.DataFrame(np.transpose(np.array([np.array(list_cluster_1),dist_1_1])),columns=['index','distance'])



    df_0_srt_slc = df_0.sort_values(by=['distance'],ascending=True).iloc[0:30,:]
    df_1_srt_slc = df_1.sort_values(by=['distance'],ascending=True).iloc[0:30,:]

    sel_indx_0_cls = df_0_srt_slc.iloc[:,0].to_numpy().flatten().astype(int)
    sel_indx_1_cls = df_1_srt_slc.iloc[:,0].to_numpy().flatten().astype(int)

 



    y_train_read_0 = y_train[sel_indx_0_cls]
    y_train_read_1 = y_train[sel_indx_1_cls]


 





    pol_thrsh = 0.5 


    read_labels_0 = y_train_read_0.to_numpy().flatten()   
    read_labels_1 = y_train_read_1.to_numpy().flatten()  

    pol_num_0 = math.floor(pol_thrsh*read_labels_0.size)     
    pol_num_1 = math.floor(pol_thrsh*read_labels_1.size)     

    pol_cnt_c0_0 = 0                    
    pol_cnt_c1_0 = 0                   

    for i in np.arange(0,read_labels_0.size):
        if read_labels_0[i] == 0:
            pol_cnt_c0_0 = pol_cnt_c0_0 + 1


    if pol_cnt_c0_0 > pol_num_0:
        clus_0 = 0
    else:
        clus_0 = 1



    for i in np.arange(0,read_labels_1.size):
        if read_labels_1[i] == 0:
            pol_cnt_c1_0 = pol_cnt_c1_0 + 1


    if pol_cnt_c1_0 > pol_num_1:
        clus_1 = 0
    else:
        clus_1 = 1






    y_train_pred = np.zeros(X_train.shape[0])

    for i in np.arange(0,X_train.shape[0]):
        if y_train_pred_cluster[i] == 0:
            y_train_pred[i] = clus_0
        else:
            y_train_pred[i] = clus_1

    y_train_pred = y_train_pred.astype(int)



    mis = 0
    for l in np.arange(0,X_train.shape[0]):
        if y_train_pred[l] != y_train.iloc[l]:
            mis = mis + 1

    train_error = (mis/X_train.shape[0])*100
    
    Train_Error.iloc[mc,0] = train_error



    confusion_matrix_train = confusion_matrix(y_train,y_train_pred)




    confusion_matrix_train_df = pd.DataFrame(confusion_matrix_train,index=['Actually 0','Actually 1'],columns=['Predicted 0','Predicted 1'])



    precision_train = (confusion_matrix_train[1][1]/(confusion_matrix_train[1][1]+confusion_matrix_train[0][1]))*100
    recall_train = (confusion_matrix_train[1][1]/(confusion_matrix_train[1][1]+confusion_matrix_train[1][0]))*100

    Precision_Train.iloc[mc,0] = precision_train
    Recall_Train.iloc[mc,0] = recall_train



    accuracy_train = ((confusion_matrix_train[0][0]+confusion_matrix_train[1][1])/(confusion_matrix_train[0][0]+confusion_matrix_train[1][1]+confusion_matrix_train[0][1]+confusion_matrix_train[1][0]))*100
    f1_train = ((2*(precision_train/100)*(recall_train/100))/((precision_train/100)+(recall_train/100))) 
    
    Accuracy_Train.iloc[mc,0] = accuracy_train
    F1_Train.iloc[mc,0] = f1_train


    y_test_pred_cluster = kmeans.predict(X_test)

 

    y_test_pred = np.zeros(X_test.shape[0])
    for i in np.arange(0,X_test.shape[0]):
        if y_test_pred_cluster[i] == 0:
            y_test_pred[i] = clus_0
        else:
            y_test_pred[i] = clus_1

    y_test_pred = y_test_pred.astype(int)



    mis = 0
    for l in np.arange(0,X_test.shape[0]):
        if y_test_pred[l] != y_test.iloc[l]:
            mis = mis + 1

    test_error = (mis/X_test.shape[0])*100
    
    Test_Error.iloc[mc,0] = test_error



    confusion_matrix_test = confusion_matrix(y_test,y_test_pred)




    confusion_matrix_test_df = pd.DataFrame(confusion_matrix_test,index=['Actually 0','Actually 1'],columns=['Predicted 0','Predicted 1'])



    precision_test = (confusion_matrix_test[1][1]/(confusion_matrix_test[1][1]+confusion_matrix_test[0][1]))*100
    recall_test = (confusion_matrix_test[1][1]/(confusion_matrix_test[1][1]+confusion_matrix_test[1][0]))*100

    Precision_Test.iloc[mc,0] = precision_test
    Recall_Test.iloc[mc,0] = recall_test


    accuracy_test = ((confusion_matrix_test[0][0]+confusion_matrix_test[1][1])/(confusion_matrix_test[0][0]+confusion_matrix_test[1][1]+confusion_matrix_test[0][1]+confusion_matrix_test[1][0]))*100
    f1_test = ((2*(precision_test/100)*(recall_test/100))/((precision_test/100)+(recall_test/100))) # divided by 100 as precision and recall specified in percentage

    Accuracy_Test.iloc[mc,0] = accuracy_test
    F1_Test.iloc[mc,0] = f1_test



    pol_thrsh_ra_range = np.arange(0,1.01,0.01)  
    ROC_df = pd.DataFrame(index=pol_thrsh_ra_range,columns=['FPR','TPR'])
    ROC_df_test = pd.DataFrame(index=pol_thrsh_ra_range,columns=['FPR','TPR'])


    for ra in np.arange(0,pol_thrsh_ra_range.size):

        pol_thrsh_ra = pol_thrsh_ra_range[ra]
        pol_num_0_ra = math.floor(pol_thrsh_ra*read_labels_0.size)    
        pol_num_1_ra = math.floor(pol_thrsh_ra*read_labels_1.size)    

        pol_cnt_c0_0_ra = 0                    
        pol_cnt_c1_0_ra = 0                    


        for i in np.arange(0,read_labels_0.size):
            if read_labels_0[i] == 0:
                pol_cnt_c0_0_ra = pol_cnt_c0_0_ra + 1


        if pol_cnt_c0_0_ra > pol_num_0_ra:
            clus_0_ra = 0
        else:
            clus_0_ra = 1



        for i in np.arange(0,read_labels_1.size):
            if read_labels_1[i] == 0:
                pol_cnt_c1_0_ra = pol_cnt_c1_0_ra + 1


        if pol_cnt_c1_0_ra > pol_num_1_ra:
            clus_1_ra = 0
        else:
            clus_1_ra = 1





        y_train_pred_ra = np.zeros(X_train.shape[0])

        for i in np.arange(0,X_train.shape[0]):
            if y_train_pred_cluster[i] == 0:
                y_train_pred_ra[i] = clus_0_ra
            else:
                y_train_pred_ra[i] = clus_1_ra

        y_train_pred_ra = y_train_pred_ra.astype(int)








        y_test_pred_ra = np.zeros(X_test.shape[0])
        for i in np.arange(0,X_test.shape[0]):
            if y_test_pred_cluster[i] == 0:
                y_test_pred_ra[i] = clus_0_ra
            else:
                y_test_pred_ra[i] = clus_1_ra

        y_test_pred_ra = y_test_pred_ra.astype(int)



        confusion_matrix_train_ra = confusion_matrix(y_train,y_train_pred_ra)
        ROC_df.iloc[ra,0] = confusion_matrix_train_ra[0][1]/(confusion_matrix_train_ra[0][1]+confusion_matrix_train_ra[0][0])
        ROC_df.iloc[ra,1] = confusion_matrix_train_ra[1][1]/(confusion_matrix_train_ra[1][1]+confusion_matrix_train_ra[1][0])


        confusion_matrix_test_ra = confusion_matrix(y_test,y_test_pred_ra)
        ROC_df_test.iloc[ra,0] = confusion_matrix_test_ra[0][1]/(confusion_matrix_test_ra[0][1]+confusion_matrix_test_ra[0][0])
        ROC_df_test.iloc[ra,1] = confusion_matrix_test_ra[1][1]/(confusion_matrix_test_ra[1][1]+confusion_matrix_test_ra[1][0])







    AUC_train = metrics.auc(ROC_df['FPR'],ROC_df['TPR'])
    AUC_Train.iloc[mc,0] = AUC_train  

    AUC_test = metrics.auc(ROC_df_test['FPR'],ROC_df_test['TPR'])
    AUC_Test.iloc[mc,0] = AUC_test
    
    
    
print('The Precision for Train for the Monte Carlo iterations is : \n',Precision_Train)
print('The Recall for Train for the Monte Carlo iterations is : \n',Recall_Train)
print('The Accuracy for Train for the Monte Carlo iterations is : \n',Accuracy_Train)
print('The F1 for Train for the Monte Carlo iterations is : \n',F1_Train)
print('The training error for the Monte Carlo iterations is : \n',Train_Error)
print('The AUC for Train for the Monte Carlo iterations is : \n',AUC_Train)



print('The Precision for Test for the Monte Carlo iterations is : \n',Precision_Test)
print('The Recall for Test for the Monte Carlo iterations is : \n',Recall_Test)
print('The Accuracy for Test for the Monte Carlo iterations is : \n',Accuracy_Test)
print('The F1 for Test for the Monte Carlo iterations is : \n',F1_Test)
print('The test error for the Monte Carlo iterations is : \n',Test_Error)
print('The AUC for Test for the Monte Carlo iterations is : \n',AUC_Test)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
The Precision for Train for the Monte Carlo iterations is : 
    Precision Train
1          95.3488
2          94.5736
3           94.964
4          95.3846
5          92.8058
6          97.5207
7          95.0355
8          93.4307
9          96.7742
10          96.063
11         94.6154
12         96.1538
13         98.3333
14         94.5312
15         96.7742
16         92.9078
17         96.9697
18          94.697
19         96.7742
20         96.0938
21            98.4
22         97.7778
23         94.1176
24         96.9231
25         96.6942
26          96.063
27         96.7213
28              96
29         97.6562
30         95.5224
The Recall for Train for the Monte Carlo iterations is : 
    Recall Train
1       72.7811
2       72.1893
3       78.1065
4       73.3728
5       76.3314
6       69.8225
7       79.2899
8       75.7396
9       71.0059
10      72.1893
11      72.7811
12      73.9645
13

In [13]:
ovrl_precision_train = Precision_Train.mean(axis=0)
ovrl_recall_train = Recall_Train.mean(axis=0)
ovrl_accuracy_train = Accuracy_Train.mean(axis=0)
ovrl_F1_train = F1_Train.mean(axis=0)
ovrl_auc_train = AUC_Train.mean(axis=0)
ovrl_train_error = Train_Error.mean(axis=0)


ovrl_precision_test = Precision_Test.mean(axis=0)
ovrl_recall_test = Recall_Test.mean(axis=0)
ovrl_accuracy_test = Accuracy_Test.mean(axis=0)
ovrl_F1_test = F1_Test.mean(axis=0)
ovrl_auc_test = AUC_Test.mean(axis=0)
ovrl_test_error = Test_Error.mean(axis=0)


print('FOR TRAIN \n')

print('\nThe overall train precision is : \n',pd.DataFrame(ovrl_precision_train).iloc[0,0],'%')
print('\nThe overall train recall is : \n',pd.DataFrame(ovrl_recall_train).iloc[0,0],'%')
print('\nThe overall train accuracy is : \n',pd.DataFrame(ovrl_accuracy_train).iloc[0,0],'%')
print('\nThe overall train F1 is : \n',pd.DataFrame(ovrl_F1_train).iloc[0,0])
print('\nThe overall train AUC is : \n',pd.DataFrame(ovrl_auc_train).iloc[0,0])
print('\nThe overall train error is : \n',pd.DataFrame(ovrl_train_error).iloc[0,0],'%')

print('\nFOR TEST \n')

print('\nThe overall test precision is : \n',pd.DataFrame(ovrl_precision_test).iloc[0,0],'%')
print('\nThe overall test recall is : \n',pd.DataFrame(ovrl_recall_test).iloc[0,0],'%')
print('\nThe overall test accuracy is : \n',pd.DataFrame(ovrl_accuracy_test).iloc[0,0],'%')
print('\nThe overall test F1 is : \n',pd.DataFrame(ovrl_F1_test).iloc[0,0])
print('\nThe overall test AUC is : \n',pd.DataFrame(ovrl_auc_test).iloc[0,0])
print('\nThe overall test error is : \n',pd.DataFrame(ovrl_test_error).iloc[0,0],'%')



print('\nNOTE : All this has been averaged over 30 Monte-Carlo Trials, with train and test selected randomly \n')

print('\nNOTE : All values except F1 Score, AUC are in %')


FOR TRAIN 


The overall train precision is : 
 95.85423075057649 %

The overall train recall is : 
 73.47140039447734 %

The overall train accuracy is : 
 88.92804698972097 %

The overall train F1 is : 
 0.8314129095274638

The overall train AUC is : 
 0.8556763209799649

The overall train error is : 
 11.071953010279001 %

FOR TEST 


The overall test precision is : 
 95.2475695269759 %

The overall test recall is : 
 72.48062015503875 %

The overall test accuracy is : 
 88.31884057971016 %

The overall test F1 is : 
 0.8204590334879324

The overall test AUC is : 
 0.8468292420327305

The overall test error is : 
 11.681159420289855 %

NOTE : All this has been averaged over 30 Monte-Carlo Trials, with train and test selected randomly 


NOTE : All values except F1 Score, AUC are in %


Again, all this is for Normalized dataset, with Unsupervised Learning using KMeans, with 30 Monte Carlo iterations